In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from newspaper import Article
import re
import nltk
import csv
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english')) 

In [ ]:
read = pd.read_csv('TestCollectionTop100Bert.csv', delimiter=',', header=None, names=["News", "Table", "Match", "Score", "Rank"])

In [ ]:
test_articles = pd.read_csv('test_articles_dataset_newyork.csv', delimiter=',')

In [ ]:
count = 0

for i, row in tqdm(test_articles.iterrows()):
    
    article_url = row['article_url']
    
    for j, rowj in read.iterrows():
    
        article_top100 = rowj['News']
        
        if article_url == article_top100:
            print(article_url)
            print(article_top100)
            count = count + 1

In [ ]:
count

In [ ]:
resultao = count/100

In [ ]:
resultao

In [ ]:
def remove_stopwords(example_sent):
    
    word_tokens = word_tokenize(example_sent) 
  
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  
    filtered_sentence = [] 
  
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    
    formattedText = ""

    for word in filtered_sentence:
        
        if (len(word)>2):
            
            formattedText = formattedText + " " +word
    
    
    formattedText = formattedText.lstrip()
    
    formattedText = formattedText.rstrip()
        
    return formattedText.lower()

In [ ]:
def clear_string(text):
    
    text = re.sub('[^A-Za-z]+',' ',text)
    
    text = text.lstrip()
    
    text = text.rstrip()
    
    return text

In [ ]:
def article_parse(url):
    
    article = Article(url)
    
    article.download()
    article.parse()
    article.nlp()
    
    title = remove_stopwords(clear_string(article.title))
    #full_text = remove_stopwords(clear_string(article.text))
    meta_description = remove_stopwords(clear_string(article.meta_description))
    summary = remove_stopwords(clear_string(article.summary))
    
    #get the list of keywords
    keywords = article.keywords
    aux1 = ''
    for word in keywords:
        
        aux1 = aux1 +" "+word
        
    keywords = remove_stopwords(clear_string(aux1))
    
    return title, meta_description, summary, keywords

In [ ]:
dataset = []

for i, row in tqdm(read[0:10].iterrows()):
    
    try:
        
        article_url = row['News']
        table_url = row['Table']
        match = row['Match']
        rank = row['Rank']
    
        news_title, news_meta_description, news_summary, news_keywords = article_parse(article_url)
        
        table_title, table_meta_description, table_summary, table_keywords = article_parse(table_url)
        
        line =     {"article_title":news_title,
                    "article_meta_description":news_meta_description,
                    "article_summary":news_summary,
                    "article_keywords":news_keywords,
                    
                    "table_title": table_title,
                    "table_meta_description": table_meta_description,
                    "table_summary": table_summary,
                    "table_keywords": table_keywords,
                    
                    "match":match,
                    "old_rank":rank}
            
        dataset.append(line)
         
    except:
        
        continue         

In [ ]:
df_dataset = pd.DataFrame(dataset)

In [ ]:
df_dataset.to_csv('dataset_top100_bert.csv',index=False)